In [2]:
# data comes from http://web.stanford.edu/~ericyi/project_page/part_annotation/index.html
from pyntcloud import PyntCloud
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
import numpy as np

import h5py

import sys
import os
if sys.platform == 'darwin':
    data_path = os.getcwd() + "/h5dataset"
else:
    data_path = os.getcwd() + "\\h5dataset"

In [3]:
h5_data = h5py.File(os.path.join(data_path, 'big_shuffled_data.h5'), mode='r')

data = h5_data.get("voxels")
label = h5_data.get("labels")
label_ref = h5_data.get("label_ref")

print(data.shape, label.shape, label_ref.value)

(29147, 32, 32, 32, 1) (29147, 5) [b'car' b'lamp' b'table' b'airplane' b'chair']


In [4]:
class cross_validation:
    
    def __init__(self, data, label, fold=5):
        self.data = np.asarray(data)
        self.label = np.asarray(label)
        assert(len(self.data) == len(self.label))
        self.fold = fold
        self.current_itr = 0;
        self.sample_in_fold = int(len(data)/fold)

    def next_bunch(self):
        """
        Return x_train, y_train, x_test, y_test
        """
        assert(self.current_itr < self.fold)
        start = self.current_itr * self.sample_in_fold
        end = (self.current_itr + 1) * self.sample_in_fold
        self.current_itr += 1
        if start == 0:
            return self.data[end:], self.label[end:], self.data[:end], self.label[:end]
        if end >= len(self.data):
            return self.data[:start], self.label[:start], self.data[start:], self.label[start:]

        return np.concatenate((self.data[:start],self.data[end:])), np.concatenate((self.label[:start],self.label[end:])), self.data[start : end], self.label[start : end]

    def have_next(self):
        if self.current_itr != self.fold:
            return True
        else:
            return False
    

In [45]:
import tensorflow as tf
def cnn_model(x_train_data, label_size, keep_rate=0.7, training=True, seed=None):
    
    if seed is not None:
        tf.set_random_seed(seed)
    
    with tf.name_scope("layer_a"):
        # conv => 32*32*32
        conv1 = tf.layers.conv3d(inputs=x_train_data, filters=16, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv1", reuse=tf.AUTO_REUSE)
        # conv => 32*32*32
        conv2 = tf.layers.conv3d(inputs=conv1, filters=32, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv2", reuse=tf.AUTO_REUSE)
        # pool => 16*16*16
        pool3 = tf.layers.max_pooling3d(inputs=conv2, pool_size=[2, 2, 2], strides=2, name="pool3")
        
    with tf.name_scope("layer_b"):
        # conv => 16*16*16
        conv4 = tf.layers.conv3d(inputs=pool3, filters=64, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv4", reuse=tf.AUTO_REUSE)
        # conv => 16*16*16
        conv5 = tf.layers.conv3d(inputs=conv4, filters=128, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv5", reuse=tf.AUTO_REUSE)
        # pool => 8*8*8
        pool6 = tf.layers.max_pooling3d(inputs=conv5, pool_size=[2, 2, 2], strides=2, name="pool6")
        
    with tf.name_scope("layer_c"):
        # conv => 8*8*8
        conv7 = tf.layers.conv3d(inputs=pool6, filters=256, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv7", reuse=tf.AUTO_REUSE)
        # conv => 8*8*8
        conv8 = tf.layers.conv3d(inputs=conv7, filters=512, kernel_size=[3,3,3], padding='same', activation=tf.nn.relu, name="conv8", reuse=tf.AUTO_REUSE)
        # pool => 4*4*4
        pool9 = tf.layers.max_pooling3d(inputs=conv8, pool_size=[2, 2, 2], strides=2, name="pool9")
        
    with tf.name_scope("batch_norm"):
        cnn3d_bn = tf.layers.batch_normalization(inputs=pool9, training=training, name="bn", reuse=tf.AUTO_REUSE)
        
    with tf.name_scope("fully_con"):
        flattening = tf.reshape(cnn3d_bn, [-1, 4*4*4*512])
        dense = tf.layers.dense(inputs=flattening, units=1024, activation=tf.nn.relu, name="full_con", reuse=tf.AUTO_REUSE)
        # (1-keep_rate) is the probability that the node will be kept
        dropout = tf.layers.dropout(inputs=dense, rate=keep_rate, training=training, name="dropout")
        
    with tf.name_scope("y_conv"):
        y_conv = tf.layers.dense(inputs=dropout, units=label_size, name="y_pred", reuse=tf.AUTO_REUSE)
    
    return y_conv

In [46]:
def get_device_name(using_gpu):
    if using_gpu:
        device_name = '/gpu:1' 
    else:
        device_name = '/cpu:0'
    return device_name


def set_placeholders(x_shape, y_shape):
  
    with tf.name_scope('inputs'):
        x_input = tf.placeholder(tf.float32, shape=x_shape, name="x_input")
        y_input = tf.placeholder(tf.float32, shape=y_shape, name="y_input")
      
    return {'x_input': x_input, 'y_input': y_input}


def get_measurement(placeholders, seed=None, keep_rate=0.5, learning_rate=0.01, training=True, using_gpu=False):
    
    x_input = placeholders['x_input']
    y_input = placeholders['y_input']
    
    device_name = get_device_name(using_gpu)
        
    with tf.device(device_name):
        
        prediction = cnn_model(x_input, y_input.shape[1].value, keep_rate, training=training, seed=seed)
        tf.add_to_collection("logits", prediction)
        
        with tf.name_scope("cross_entropy"):
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y_input), name="cross_entropy")
                              
        with tf.name_scope("training"):
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
            tf.add_to_collection("optimizer", optimizer)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_input, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name="acc")    
    
    return {'prediction': prediction, 'cost': cost, 'optimizer': optimizer, 'accuracy': accuracy}

In [58]:
def run_nn(data, label, placeholders, measurments, summary_op=None, batch_size=32, epoch_step=0, training=True, using_gpu=False):
    
    x_input = placeholders['x_input']
    y_input = placeholders['y_input']
    
    prediction = measurments['prediction']
    cost = measurments['cost'] 
    optimizer = measurments['optimizer'] 
    accuracy = measurments['accuracy'] 
    
    iterations = int(len(data)/batch_size) + 1
    
    summary_step = 8
    summary_counter = int(epoch_step*(len(data)/summary_step))
    
    acc = 0
    
    # mini batch
    for itr in range(iterations):
        mini_batch_x = data[itr*batch_size: (itr+1)*batch_size]
        mini_batch_y = label[itr*batch_size: (itr+1)*batch_size]
        
        if training and summary_op is None:
            _optimizer, _cost = sess.run([optimizer, cost], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
            print('\tLost for', itr + 1, "/", iterations, _cost, end='\r')
            
        elif training and summary_op is not None:
            _optimizer, _cost, _summary = sess.run([optimizer, cost, summary_op['summary']], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
            print('\tLost for', itr + 1, "/", iterations, _cost, end='\r')
            summary_op['train_writer'].add_summary(_summary, summary_counter)
            summary_op['train_writer'].flush()
            
        elif not training and summary_op is not None:
            _acc, _summary = sess.run([accuracy, summary_op['summary']], feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
            print('\tAccuacy for', itr + 1, "/", iterations, _acc, end='\r')
            acc += _acc
            summary_op['test_writer'].add_summary(_summary, summary_counter)
            summary_op['test_writer'].flush()
            
        else:
            _acc = sess.run(accuracy, feed_dict={x_input: mini_batch_x, y_input: mini_batch_y})
            print('\tAccuacy for', itr + 1, "/", iterations, _acc, end='\r')
            acc += _acc
        
        summary_counter += int(batch_size/summary_step)

    print("\n")
    
    if not training:
        print("\tTesting Accuracy:", acc/iterations)
            

In [54]:
def train_neural_network(x_train_data, y_train_data, x_test_data, y_test_data, placeholders, measurments, summary_op=None, epochs=10, batch_size=128, save_model=False, model_name=None, using_gpu=False):

    import datetime
    start_time = datetime.datetime.now()

    # run epochs
    for epoch in range(epochs):
        start_time_epoch = datetime.datetime.now()
        print('Epoch', epoch, 'started')

        # train
        run_nn(x_train_data, y_train_data, placeholders, measurments, summary_op=summary, batch_size=batch_size, epoch_step=epoch, training=True, using_gpu=using_gpu)

         # test
        run_nn(x_test_data, y_test_data, placeholders, measurments, summary_op=summary, batch_size=batch_size, epoch_step=epoch, training=False, using_gpu=using_gpu)

        end_time_epoch = datetime.datetime.now()
        print('\tTime elapse:', str(end_time_epoch - start_time_epoch)) 

        if save_model:
            import os
            if model_name is not None:
                model_path = os.path.join(os.getcwd(), 'model', model_name)
            else:
                model_path = os.path.join(os.getcwd(), 'model', 'model')
            saver = tf.train.Saver()
            saver.save(sess, model_path, global_step=epoch)

    end_time = datetime.datetime.now()
    print('Time elapse: ', str(end_time - start_time))

In [9]:
cv = cross_validation(data, label, fold=5)

_x_train, _y_train, _x_test, _y_test = cv.next_bunch()

In [59]:
tf.reset_default_graph()
    
x_shape=[None, 32, 32, 32, 1]
y_shape=[None, 5]
    
using_gpu=True
record_performance=True

config = None
if using_gpu:
    # GPU using BFC
    config = tf.ConfigProto()
    config.gpu_options.allocator_type = 'BFC'
    
with tf.Session(config=config) as sess:
    
    placeholders = set_placeholders(x_shape, y_shape)
    
    measurments = get_measurement(placeholders, keep_rate=0.5, seed=1234, training=True, learning_rate=0.005, using_gpu=using_gpu)
    
    summary = None
    if record_performance:
        tf.summary.scalar("cross_entropy", measurments['cost'])
        tf.summary.scalar("accuracy", measurments['accuracy'])
        summary_all = tf.summary.merge_all()
        logs_path = os.path.join(os.getcwd(), 'summaries')
        train_writer = tf.summary.FileWriter(os.path.join(logs_path, 'train'), graph=tf.get_default_graph())
        test_writer = tf.summary.FileWriter(os.path.join(logs_path, 'test'), graph=tf.get_default_graph())
        summary = {'summary': summary_all, 'train_writer': train_writer, 'test_writer': test_writer}
    
    sess.run(tf.global_variables_initializer())
    
    train_neural_network(_x_train, _y_train, _x_test, _y_test, placeholders, measurments, summary_op=summary, epochs=2, batch_size=32, using_gpu=using_gpu)
    
    summary['train_writer'].close()
    summary['test_writer'].close()
    

Epoch 0 started
	Lost for 729 / 729 0.11849122

	Accuacy for 183 / 183 1.00625

	Testing Accuracy: 0.932206284153
	Time elapse: 0:10:57.892431
Epoch 1 started
	Lost for 729 / 729 0.00696487

	Accuacy for 183 / 183 0.86875

	Testing Accuracy: 0.918989071103
	Time elapse: 0:10:59.539337
Time elapse:  0:21:57.431768


# Grid Search

In [ ]:
x_shape=[None, 32, 32, 32, 1]
y_shape=[None, 5]
    
using_gpu=True

config = None

if using_gpu:
    # GPU using BFC
    config = tf.ConfigProto()
    config.gpu_options.allocator_type = 'BFC'
tf.reset_default_graph()    


learning_rates = [0.1, 0.01, 0.005, 0.001, 0.0005]
batch_sizes = [16, 32, 64]
keep_rates = [0.5, 0.7]

for lr in learning_rates:
    for bs in batch_sizes:
        for kr in keep_rates:
            
            tf.reset_default_graph()
            with tf.Session(config=config) as sess:
                placeholders = set_placeholders(x_shape, y_shape)
                measurments = get_measurement(placeholders, keep_rate=kr, seed=1234, training=True, learning_rate=lr, using_gpu=using_gpu)

                tf.summary.scalar("cross_entropy", measurments['cost'])
                tf.summary.scalar("accuracy", measurments['accuracy'])
                summary_all = tf.summary.merge_all()
                logs_path = os.path.join(os.getcwd(), 'summaries')
                filename_suffix = "_keep_rate_"+str(kr)+"_batch_sizes_"+str(bs)+"_learning_rates_"+str(lr)
                train_writer = tf.summary.FileWriter(os.path.join(logs_path, 'train'), graph=tf.get_default_graph(), filename_suffix=filename_suffix)
                test_writer = tf.summary.FileWriter(os.path.join(logs_path, 'test'), graph=tf.get_default_graph(), filename_suffix=filename_suffix)
                summary = {'summary': summary_all, 'train_writer': train_writer, 'test_writer': test_writer}

                sess.run(tf.global_variables_initializer())

                train_neural_network(_x_train[:], _y_train[:], _x_test[:], _y_test[:], placeholders, measurments, summary_op=summary, epochs=5, batch_size=bs, using_gpu=using_gpu)

                summary['train_writer'].close()
                summary['test_writer'].close()


Epoch 0 started
	Lost for 1458 / 1458 1.77165

	Accuacy for 365 / 365 0.4755 365 0.375 / 365 0.1875 / 365 0.25 / 365 0.4375 / 365 0.3125 / 365 0.1875 / 365 0.25

	Testing Accuracy: 0.267020547962
	Time elapse: 0:12:07.779207
Epoch 1 started
	Lost for 1458 / 1458 1.77388

	Accuacy for 365 / 365 0.4755/ 365 0.25 / 365 0.125 / 365 0.25/ 365 0.25 / 365 0.25 / 365 0.25 / 365 0.0

	Testing Accuracy: 0.267020547962
	Time elapse: 0:12:01.219626
Epoch 2 started
	Lost for 1458 / 1458 1.77358

	Accuacy for 365 / 365 0.4755/ 365 0.25 / 365 0.25 / 365 0.3125 / 365 0.125 / 365 0.25 / 365 0.125 / 365 0.375 / 365 0.3125340 / 365 0.3125

	Testing Accuracy: 0.267020547962
	Time elapse: 0:12:03.228097
Epoch 3 started
	Lost for 1458 / 1458 1.77427

	Accuacy for 365 / 365 0.4875 / 365 0.0 / 365 0.1875 / 365 0.25 / 365 0.3125/ 365 0.375 / 365 0.375

	Testing Accuracy: 0.267020547962
	Time elapse: 0:12:04.101836
Epoch 4 started
	Lost for 1458 / 1458 1.77442

	Accuacy for 365 / 365 0.4755/ 365 0.25 / 365 0.18

	Lost for 729 / 729 1.5679287

	Accuacy for 183 / 183 0.48125

	Testing Accuracy: 0.267383879814
	Time elapse: 0:11:01.434724
Time elapse:  0:55:06.103856
Epoch 0 started
	Lost for 729 / 729 0.024613873

	Accuacy for 183 / 183 1.00625

	Testing Accuracy: 0.922643442623
	Time elapse: 0:11:00.784710
Epoch 1 started
	Lost for 729 / 729 0.109385578

	Accuacy for 183 / 183 1.00625

	Testing Accuracy: 0.951673497268
	Time elapse: 0:11:00.752515
Epoch 2 started
	Lost for 729 / 729 0.033339538

	Accuacy for 183 / 183 1.06875

	Testing Accuracy: 0.947404371585
	Time elapse: 0:11:02.554525
Epoch 3 started
	Lost for 729 / 729 0.109751615

	Accuacy for 183 / 183 0.80625

	Testing Accuracy: 0.936065573836
	Time elapse: 0:11:00.135485
Epoch 4 started
	Lost for 729 / 729 0.277085396

	Accuacy for 183 / 183 0.80625

	Testing Accuracy: 0.783060109355
	Time elapse: 0:11:01.419902
Time elapse:  0:55:05.647137
Epoch 0 started
	Lost for 365 / 365 0.1191652

	Accuacy for 92 / 92 1.037575

	Testing Accuracy:

In [ ]:
learning_rates = [0.1, 0.01, 0.005, 0.001, 0.0005]
batch_sizes = [16, 32, 64]
keep_rates = [0.5, 0.7]


for lr in learning_rates:
    for bs in batch_sizes:
        for kr in keep_rates:
            tf.reset_default_graph()
            train_neural_network(_x_train[:], _y_train[:], _x_test[:], _y_test[:], learning_rate=lr, batch_size=bs, keep_rate=kr, epochs=1, using_gpu=True)